In [1]:
from requests_html import HTMLSession
import pandas as pd

## アメリカ以外

In [2]:
url = 'https://ja.wikipedia.org/wiki/%E5%9B%BD%E9%9A%9B%E7%A9%BA%E6%B8%AF%E3%81%AE%E4%B8%80%E8%A6%A7'

In [3]:
session = HTMLSession()
r = session.get(url)

In [4]:
tables = r.html.find('.mw-parser-output .wikitable')

In [5]:
headlines = r.html.find('.mw-parser-output .mw-headline')

In [6]:
remove_names = ['アメリカ合衆国', 'フランス', 'アンドラ', 'サンマリノ', 'バチカン', 'リヒテンシュタイン', '関連項目']
ext_headlines = []
for h in headlines:
    if len(h.text)==1:
        continue
    
    if h.text in remove_names:
        continue
    
    ext_headlines.append(h)
#     print(h.text)

In [7]:
print(len(tables))
print(len(ext_headlines))

199
199


In [8]:
infos = []

for table, country in zip(tables, ext_headlines):
    country_name = country.text
    trs = table.find('tr')[1:]
    for tr in trs:
        th = tr.find('th', first=True)
        tds = tr.find('td')

        ja_name = th.text
        en_name = tds[0].text
        area_name = tds[2].text
        iata = tds[3].text
        icao = tds[4].text
        
        info = [country_name, ja_name, en_name, area_name, iata, icao]
        infos.append(info)

In [70]:
df = pd.DataFrame(infos)

## アメリカ

In [73]:
url = 'https://ja.wikipedia.org/wiki/%E3%82%A2%E3%83%A1%E3%83%AA%E3%82%AB%E5%90%88%E8%A1%86%E5%9B%BD%E3%81%AE%E7%A9%BA%E6%B8%AF%E3%81%AE%E4%B8%80%E8%A6%A7'
r = session.get(url)

In [97]:
tables = r.html.find("table[border='0']")

In [90]:
headlines = r.html.find('.mw-headline')

In [101]:
rm_headline = ['注釈', '参考文献', 'デラウェア州']
ext_headlines = []
for h in headlines:
    if h.text in rm_headline:
        continue
        
    ext_headlines.append(h)

In [103]:
infos = []
country_name = 'アメリカ合衆国'
for table, headline in zip(tables, headlines):
    trs = table.find('tr')[1:]
    area_name = headline.text
    for tr in trs:
        tds = tr.find('td')
        ja_name = tds[0].text
        iata = tds[1].text
        icao = tds[2].text
        en_name = tds[3].text
        
        info =  [country_name, ja_name, en_name, area_name, iata, icao]
        infos.append(info)

## 結合

In [104]:
df_us = pd.DataFrame(infos)
df_all = pd.concat([df, df_us])
df_all.columns = ['country_name', 'ja_name', 'en_name', 'area_name', 'iata', 'icao']

In [116]:
df_all.to_clipboard(sep='-')